In [89]:
from google.colab import drive
drive.mount('/content/drive') 

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
!mkdir /usr/lib/python3.7/metrics
!cp -R /content/drive/MyDrive/BA/Notebooks/2_Experiment/review_object_detection_metrics-main/src /usr/lib/python3.7/metrics/src

In [4]:
!pip install PyQt5
!pip install -qU torch_snippets

     |████████████████████████████████| 8.3 MB 12.8 MB/s 
     |████████████████████████████████| 338 kB 61.4 MB/s 
     |████████████████████████████████| 59.9 MB 1.2 MB/s 
     |████████████████████████████████| 45 kB 1.9 MB/s 
     |████████████████████████████████| 948 kB 30.2 MB/s 
     |████████████████████████████████| 10.9 MB 59.4 MB/s 
     |████████████████████████████████| 58 kB 5.5 MB/s 
     |████████████████████████████████| 60 kB 6.4 MB/s 
     |████████████████████████████████| 229 kB 70.6 MB/s 
     |████████████████████████████████| 78 kB 6.7 MB/s 
     |████████████████████████████████| 51 kB 5.5 MB/s 
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow 2.8.0 requires tf-estimator-nightly==2.8.0.dev2021122109, which is not installed.
arviz 0.11.4 requires typing-extensions<4,>=3.7.4.3, but you have typing-extensions 4.1.1 which is inco

In [25]:
import copy
import glob
import torch
import time
import statistics
import pandas as pd
import numpy as np

from IPython import display
from torch_snippets import *
from os.path import join
from PIL import Image
from metrics.src.bounding_box import BoundingBox
from metrics.src.evaluators import coco_evaluator, pascal_voc_evaluator
from metrics.src.bounding_box import BoundingBox
from metrics.src.utils.enumerators import BBFormat, BBType, CoordinatesType, MethodAveragePrecision
from skimage import data
from skimage.color import rgb2hsv, rgb2luv, rgb2lab

In [6]:
df_train = pd.read_csv('/content/drive/MyDrive/BA/dataset/Experimente/df_80_20_train_tf.csv')

In [7]:
df_train.head(2)

,filename,class,xmin,xmax,ymin,ymax
0,0000599864fd15b3.jpg,Bus,0.343750,0.90875,0.156162,0.650047
1,00006bdb1eb5cd74.jpg,Truck,0.276667,0.69750,0.141604,0.437343


In [8]:
width, height = 300, 300
IMAGE_ROOT = '/content/drive/MyDrive/BA/dataset/bus-trucks/images'

label2target = {l:t+1 for t,l in enumerate(df_train['class'].unique())}
label2target['background'] = 0
target2label = {t:l for l,t in label2target.items()}
label2target = {v: k for k, v in target2label.items()}
background_class = label2target['background']
num_classes = len(label2target)


def preprocess_image(img):
  img = torch.tensor(img).permute(2,0,1)
  return img.to(device).float()

class OpenDataset(torch.utils.data.Dataset):
  w, h = width, height
  def __init__(self, df, image_dir=IMAGE_ROOT):
    self.image_dir = image_dir
    self.files = glob.glob(self.image_dir+'/*')
    self.df = df
    self.image_infos = df['filename'].unique()

  def __getitem__(self, ix):

    #filename	class	xmin	xmax	ymin	ymax

    # # load images and masks
    image_id = self.image_infos[ix]
    img_path = find(image_id, self.files)
    img = Image.open(img_path).convert('RGB')
    img = np.array(img.resize((self.w, self.h), resample=Image.BILINEAR))/255.

    data = self.df[self.df['filename'] == image_id]
    labels = data['class'].values.tolist()
    data = data[['xmin','ymin','xmax','ymax']].values
    data[:,[0,2]] *= self.w
    data[:,[1,3]] *= self.h
    boxes = data.astype(np.uint32).tolist() # convert to absolute coordinates
    # torch FRCNN expects ground truths as a dictionary of tensors
    target = {}
    target["boxes"] = torch.Tensor(boxes).float()
    target["labels"] = torch.Tensor([label2target[i] for i in labels]).long()
    img = preprocess_image(img)

    return img, target

  def collate_fn(self, batch):
    return tuple(zip(*batch)) 

  def __len__(self):
    return len(self.image_infos)

In [10]:
df_train = df_train[:150]

In [11]:
train_ds = OpenDataset(df_train)
train_loader = torch.utils.data.DataLoader(train_ds, batch_size=1, collate_fn=train_ds.collate_fn, drop_last=True)

# Average Object size

In [12]:
bounding_boxes_train = []

for ix, (images, targets) in enumerate(train_loader):
    #Ground Truth
  for i, box in enumerate(targets[0]['boxes']):
    cls = target2label[int(targets[0]['labels'][i])]
    bb = BoundingBox(
      image_name        = str(ix),
      class_id          = cls,
      coordinates       = list(box),
      type_coordinates  = CoordinatesType.ABSOLUTE,
      bb_type           = BBType.GROUND_TRUTH,
      confidence        = None,
      format            = BBFormat.XYX2Y2
    )
    bounding_boxes_train.append(bb)
  if ix % 20 == 0:
    p = ix / len(train_loader)
    print(p, end = '\r')

In [13]:
areas = []
for bbs in bounding_boxes_train:
  areas.append(bbs.get_area())

In [14]:
from sklearn.cluster import KMeans
import numpy as np
import math

kmeans = KMeans(n_clusters=3, random_state=0).fit(np.array(areas).reshape(-1, 1))

In [15]:
x = np.linspace(min(areas), max(areas), 100000)
preds = kmeans.predict(x.reshape(-1, 1))

In [16]:
changes = np.where(preds[:-1] != preds[1:])[0]
print('Object boundaries for S and L sized objects', np.sqrt(x[changes]))

Object boundaries for S and L sized objects [144.80688123 232.3492006 ]


In [86]:
def image_light(img, thrshld = 127):
  rgb = img.cpu().permute(1,2,0) * 255
  rgb = np.dot(rgb[...,:3], [0.2989, 0.5870, 0.1140])
  
  is_light = np.mean(rgb) > thrshld

  # Light: 0
  # Dark: 1

  # return 0 if is_light else 1
  return {'value': np.mean(rgb),
          'brightness': 'light' if is_light else 'dark'
          }

def avg_img_color(img):
  rgb_avg = img.permute(1,2,0).mean(axis=(0, 1)) 
  hsv_avg = rgb2hsv(img.permute(1,2,0)).mean(axis=(0, 1))
  luv_avg = rgb2luv(img.permute(1,2,0)).mean(axis=(0, 1))
  lab_avg = rgb2lab(img.permute(1,2,0)).mean(axis=(0, 1))

  return {
    'rgb_avg_r': float(rgb_avg[0]),
    'rgb_avg_g': float(rgb_avg[1]),
    'rgb_avg_b': float(rgb_avg[2]),
    'hsv_avg_h': float(hsv_avg[0]),
    'hsv_avg_s': float(hsv_avg[1]),
    'hsv_avg_v': float(hsv_avg[2])
  }

In [132]:
def bb_intersection_over_union(boxA, boxB):

  xA = max(boxA[0], boxB[0])
  yA = max(boxA[1], boxB[1])
  xB = min(boxA[2], boxB[2])
  yB = min(boxA[3], boxB[3])

  interArea = (xB - xA) * (yB - yA)

  boxAArea = (boxA[2] - boxA[0]) * (boxA[3] - boxA[1])
  boxBArea = (boxB[2] - boxB[0]) * (boxB[3] - boxB[1])

  iou = interArea / float(boxAArea + boxBArea - interArea)

  return iou


def detected(gt, detection, iou_threshold ):

  for i, g in enumerate(gt):
    if g[0] == detection[0]:
      if g[1] == detection[1]:
        iuo = bb_intersection_over_union(g[-4:], detection[-4:])
        if iuo >= iou_threshold:
          return True

  return False

In [133]:

metadata = []

for ix, (images, targets) in enumerate(train_loader):
  img = images[0]
  t = targets[0]['boxes']

  c_img = image_light(img)
  c_avg_img = avg_img_color(img)

  gts = []
  for i, box in enumerate(targets[0]['boxes']):
    cls = target2label[int(targets[0]['labels'][i])]
    X1, Y1, X2, Y2 = box.tolist()
    gts.append((ix, cls, 1, X1, Y1, X2, Y2))

  for i, box in enumerate(targets[0]['boxes']):
    cls = target2label[int(targets[0]['labels'][i])]

    X1, Y1, X2, Y2 = box.numpy().astype(int)

    # object frame
    obj_frame = img[:3, Y1:Y2, X1:X2]

    # colors
    c_obj = image_light(obj_frame)
    c_avg_obj = avg_img_color(obj_frame)

    # Area
    bb = BoundingBox(
      image_name        = str(ix),
      class_id          = cls,
      coordinates       = list(box),
      type_coordinates  = CoordinatesType.ABSOLUTE,
      bb_type           = BBType.GROUND_TRUTH,
      confidence        = None,
      format            = BBFormat.XYX2Y2
    )
    
    # detected
    det = detected(
        gts,
        [ix, cls, 1, X1, Y1, X2, Y2],
        0.1
    )

    metadata.append({'image_id': ix,
                   'image_brightness_value': c_img['value'],
                   'image_brightness_interpretation': c_img['brightness'],
                   'image_rgb_avg_r': c_avg_img['rgb_avg_r'],
                   'image_rgb_avg_g': c_avg_img['rgb_avg_g'],
                   'image_rgb_avg_b': c_avg_img['rgb_avg_b'],
                   'image_hsv_avg_h': c_avg_img['hsv_avg_h'],
                   'image_hsv_avg_s': c_avg_img['hsv_avg_s'],
                   'image_hsv_avg_v': c_avg_img['hsv_avg_v'],
                   'object_brightness_value': c_obj['value'],
                   'object_brightness_interpretation': c_obj['brightness'],
                   'object_rgb_avg_r': c_avg_obj['rgb_avg_r'],
                   'object_rgb_avg_g': c_avg_obj['rgb_avg_g'],
                   'object_rgb_avg_b': c_avg_obj['rgb_avg_b'],
                   'object_hsv_avg_h': c_avg_obj['hsv_avg_h'],
                   'object_hsv_avg_s': c_avg_obj['hsv_avg_s'],
                   'object_hsv_avg_v': c_avg_obj['hsv_avg_v'],
                   'detected': det,
                   'object_size': bb.get_area(),
                   'object_class': cls
                   }
                  )
    break

In [1]:
df = pd.DataFrame.from_dict(metadata)
df.tail()


NameError: ignored

In [90]:
print('Image average brightness value', df['image_brightness_value'].mean())
print('Image std. brightness value', df['image_brightness_value'].std())
# print('Image average dark', len(df[df['brightness'] == 'dark']) / len(df))
# print('Image average light', len(df[df['brightness'] == 'light']) / len(df))
print('Object average brightness value', df['object_brightness_value'].mean())
print('Object std. brightness value', df['object_brightness_value'].std())

Image average brightness value 120.74817606027413
Image std. brightness value 22.478238773891274
Object average brightness value 103.76224241800544
Object std. brightness value 25.549411450476665


Objekte sind durchschnittlich dunkler